In [1]:
!pip install ijson spider-client langchain_mistralai langchain_community beautifulsoup4 faiss-cpu langchain-text-splitters sentence-transformers mistralai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached langchain_mistralai-0.2.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_community-0.3.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_text_splitters-0.3.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached langchain_core-0.3.15-py3-none-any.whl.metadata (6.3 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
  Using cached pydantic_settings-2.6.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2

## Les imports

In [3]:
import os
import pickle
from pprint import pprint

## Chargement des variables d'environnement

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [9]:
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
#SPIDER_API_KEY = os.getenv("SPIDER_API_KEY")

## Les données

### Scraping

In [7]:
#from langchain_community.document_loaders import SpiderLoader

In [13]:
"""
loader = SpiderLoader(
    api_key=SPIDER_API_KEY,
    url="https://fabeon.fr/",
    mode="crawl",  # if no API key is provided it looks for SPIDER_API_KEY in env
)
data = loader.load()
"""

'\nloader = SpiderLoader(\n    api_key=SPIDER_API_KEY,\n    url="https://fabeon.fr/",\n    mode="crawl",  # if no API key is provided it looks for SPIDER_API_KEY in env\n)\ndata = loader.load()\n'

In [14]:
#with open("fabeon.pkl", "wb") as f:
#    pickle.dump(data, f)

In [15]:
with open("fabeon.pkl", "rb") as f:  # open a text file
    data = pickle.load(f)

In [16]:
len(data)

54

In [17]:
pprint(dict(data[0]))

{'id': None,
 'metadata': {'description': "Lever les freins par l'innovation pour former "
                             'aux métiers de demain dans le domaine de '
                             'l’imprimerie numérique',
              'domain': 'fabeon.fr',
              'extracted_data': None,
              'file_size': 7659,
              'keywords': ['17h30 à 18h15 Mercredi 20 novembre',
                           '17h30 à 18h15 Mercredi 4 décembre',
                           '2024 Nombres de stagiaires',
                           '80 rue Tobias Stimmer 67400 Illkirch-Graffenstaden',
                           '> La SMART FACTORY',
                           '> Le CAMPUS',
                           'Aller au contenu',
                           'Ateliers découverte Créez',
                           'Bachelor Digital Printing',
                           'Cliquez ci-dessous pour visualiser notre '
                           'certificat Qualiopi',
                           'Copyrig

### Préparation

In [18]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [19]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [20]:
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(data[0].page_content)

In [22]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    strip_headers=False
)

In [23]:
docs = []
for doc in data:
    md_header_splits = markdown_splitter.split_text(doc.page_content)
    docs += md_header_splits

In [24]:
len(docs)

256

In [25]:
docs[2]

Document(metadata={'Header 3': 'Les dates de nos prochains ateliers découverte'}, page_content='### Les dates de nos prochains ateliers découverte\nMercredi 6 novembre (en visio) 17h30 à 18h15\nMercredi 20 novembre (en visio) 17h30 à 18h15\nMercredi 4 décembre (sur place) 17h30 à 18h15\n[Lire la suite](https://www.linkedin.com/school/91674952/admin/page-posts/published/)')

## Le machine learning

### Peuplement de la base de données vectorielle

#### Transformation des documents en vecteurs

In [28]:
import numpy as np
from sentence_transformers import SentenceTransformer

/home/jmfrmg/Projets/devia/LEX_RAG_GRAG/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [29]:
embedding_model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")

In [30]:
#embedding_model.to("cuda")

In [31]:
king_embedding = embedding_model.encode("king")
man_embedding = embedding_model.encode("man")
woman_embedding = embedding_model.encode("woman")
queen_embedding = embedding_model.encode("queen")
squirrel_embedding = embedding_model.encode("squirrel")
cup_embedding = embedding_model.encode("cup")

In [32]:
from scipy import spatial

result = 1 - spatial.distance.cosine(king_embedding, cup_embedding)

In [33]:
result

0.41686059701367717

In [34]:
1 - spatial.distance.cosine(king_embedding, queen_embedding)

0.8508266252933384

In [35]:
1 - spatial.distance.cosine(king_embedding, woman_embedding)

0.45114378011149814

In [36]:
for doc_list in docs:
    text_embeddings = np.array([embedding_model.encode(doc.page_content) for doc in docs])

In [36]:
text_embeddings.shape

(281, 512)

#### Peuplement de la base de données

In [37]:
import faiss

In [38]:
d = text_embeddings.shape[1]

In [39]:
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

### Génération augmentée

In [40]:
from mistralai import Mistral

In [41]:
llm_client = Mistral(api_key=MISTRAL_API_KEY)

In [42]:
question = "Fabéon est-il un incubateur de startups ?"
question1 = "Quelles sont les formations proposées par Fabéon ?"
question2 = "Quelles sont les dates des ateliers découverte ?"
question_embeddings = np.array([embedding_model.encode(question)])

In [45]:
# Recherche dans la base de données vectorielle ans ajouter de contexte
D, I = index.search(question_embeddings, k=3)
retrieved_chunk = "\n".join([docs[i].page_content for i in I.tolist()[0]]) 

In [46]:
# Recherche dans la base de données vectorielle avec contexte
# Les deux documents précédents et suivants sont ajoutés
D, I = index.search(question_embeddings, k=3)

retrieved_chunk = []
test_chunk = []
for i in I.tolist()[0]:
    test_chunk.append([docs[i2].page_content for i2 in range(i-2,i+3)])
    chunk = "".join([docs[i2].page_content for i2 in range(i-2,i+3)])
    retrieved_chunk.append(chunk)
retrieved_chunk = "\n".join(retrieved_chunk)

In [47]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [48]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = llm_client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [49]:
print(run_mistral(question))

Fabéon n'est pas un nom largement reconnu comme un incubateur de startups. Il est possible que Fabéon soit une organisation locale ou spécialisée qui n'a pas atteint une notoriété internationale. Les incubateurs de startups sont des structures qui offrent des ressources, des conseils, du financement et un espace de travail aux jeunes entreprises pour les aider à démarrer et à croître.

Si vous avez des informations spécifiques ou un contexte particulier concernant Fabéon, n'hésitez pas à les partager pour que je puisse vous fournir une réponse plus précise.


In [50]:
print(run_mistral(prompt))

Oui, Fabéon est un incubateur de startups. Le texte mentionne que Fabéon Campus propose un cursus professionnalisant et résolument immersif, où les apprenants côtoient des startups, des experts, et un écosystème dynamique. De plus, il est explicitement indiqué que Fabéon est situé dans un parc d'innovation qui inclut un incubateur pour accueillir les startups et les innovateurs.
